In [ ]:
## Reference : https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14703296765524404074, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7529173730629434721
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
!cat /proc/meminfo

MemTotal:       13302924 kB
MemFree:         9365996 kB
MemAvailable:   12089244 kB
Buffers:          132456 kB
Cached:          2640608 kB
SwapCached:            0 kB
Active:          1327652 kB
Inactive:        2241932 kB
Active(anon):     673828 kB
Inactive(anon):     2500 kB
Active(file):     653824 kB
Inactive(file):  2239432 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1652 kB
Writeback:             0 kB
AnonPages:        796488 kB
Mapped:           570248 kB
Shmem:              3244 kB
KReclaimable:     142936 kB
Slab:             198584 kB
SReclaimable:     142936 kB
SUnreclaim:        55648 kB
KernelStack:        5108 kB
PageTables:         8864 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    3611536 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       46800 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
## Only for saving model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import os
os.mkdir("NER_RE_New")

In [ ]:
cd /content/drive/MyDrive/NER_RE_New

/content/drive/MyDrive/NER_RE_New


In [ ]:
os.mkdir('NER')
%cd NER
#upload train and dev data

/content/drive/MyDrive/NER_RE_New/NER


# NER Training

## I/P : .tsv file in IOB format (created data from UBIAI annotation tools)

In [ ]:
!pip install -U spacy
!python -m spacy info

     |████████████████████████████████| 5.9 MB 5.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 627 kB 43.0 MB/s 
     |████████████████████████████████| 10.1 MB 36.5 MB/s 
     |████████████████████████████████| 451 kB 35.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4

============================== Info about spaCy ==============================

spaCy version    3.1.3 

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-10-24 09:47:23--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-10-24 09:47:24--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?criLEeuMLMjn74oJCGyVinoIyatSwgHJ0xSx4u0YnzDxafXMmmwe9GeIdrPsCCQorIzU497NfsXwf09RXkoe10sYxEz6CpAzMyI88XsVldpU_

In [ ]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.2 MB 5.8 kB/s 
     |████████████████████████████████| 42 kB 518 kB/s 
     |████████████████████████████████| 2.6 MB 8.8 MB/s 
     |████████████████████████████████| 998 kB 46.0 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.1 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
'''If Error: KeyError: 'packaging'  use pip install -U spacy[transformers] '''

!pip install -U spacy[cuda92,transformers]     
!export CUDA_PATH=”/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH
!pip install cupy

     |████████████████████████████████| 55.0 MB 27 kB/s 
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file
     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Created wheel for cupy: filename=cupy-9.5.0-cp37-cp37m-linux_x86_64.whl size=53827658 sha256=d7ce78354a8b3a38b921c93ec5c6d3140c27897831f8d4eb7a1137b5f2ae1e7a
  Stored in directory: /root/.cache/pip/wheels/03/cf/00/c29a65b64519d1c3952308947ccba531cefd04b3c3b50d69a0
Successfully built cupy


In [ ]:
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 577.3 MB 2.8 kB/s 
     |████████████████████████████████| 12.5 MB 232 kB/s 
     |████████████████████████████████| 7.6 MB 4.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1+cu92 which is incompatible.


In [ ]:
## Upload data .tsv files  Eg. from  https://github.com/UBIAI/Fine_tune_BERT_with_spacy3
!python -m spacy convert "/content/drive/MyDrive/NER_RE_New/NER/Dev.tsv" "/content/drive/MyDrive/NER_RE_New/NER" -t json -n 1 -c iob
!python -m spacy convert "/content/drive/MyDrive/NER_RE_New/NER/Train.tsv" "/content/drive/MyDrive/NER_RE_New/NER" -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (53 documents):
/content/drive/MyDrive/NER_RE_New/NER/Dev.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (124 documents):
/content/drive/MyDrive/NER_RE_New/NER/Train.json


In [ ]:
!python -m spacy convert "/content/drive/MyDrive/NER_RE_New/NER/Dev.json" "/content/drive/MyDrive/NER_RE_New/NER" -t spacy
!python -m spacy convert "/content/drive/MyDrive/NER_RE_New/NER/Train.json" "/content/drive/MyDrive/NER_RE_New/NER" -t spacy

✔ Generated output file (53 documents):
/content/drive/MyDrive/NER_RE_New/NER/Dev.spacy
✔ Generated output file (124 documents):
/content/drive/MyDrive/NER_RE_New/NER/Train.spacy


In [ ]:
'''If Error : KeyError: 'filenames'  
   Solution : filenames = config[lib]['filename'] '''

## download "base_config.cfg" from https://spacy.io/usage/training  and update  [paths]  section  train = /content/drive/MyDrive/NER_RE/NER/Train.spacy and dev = /content/drive/MyDrive/NER_RE/NER/Dev.spacy
   
!python -m spacy init fill-config /content/drive/MyDrive/NER_RE_New/NER/base_config.cfg /content/drive/MyDrive/NER_RE_New/NER/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/NER_RE_New/NER/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train -g 0 /content/drive/MyDrive/NER_RE_New/NER/config.cfg --output /content/drive/MyDrive/NER_RE_New/NER/output --paths.train /content/drive/MyDrive/NER_RE_New/NER/Train.spacy --paths.dev /content/drive/MyDrive/NER_RE_New/NER/Dev.spacy
#!python -m spacy train -g 0 /content/gdrive/MyDrive/config.cfg --output /content/gdrive/MyDrive/output --paths.train ./Train.spacy --paths.dev ./Dev.spacy

ℹ Saving to output directory:
/content/drive/MyDrive/NER_RE_New/NER/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-10-24 10:17:29,319] [INFO] Set up nlp object from config
[2021-10-24 10:17:29,332] [INFO] Pipeline: ['transformer', 'ner']
[2021-10-24 10:17:29,337] [INFO] Created vocabulary
[2021-10-24 10:17:29,339] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 519kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 4.11MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 2.63MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 4.56MB/s]
Downloading: 100% 501M/501M [00:14<00:00, 35.7MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a 

In [ ]:
import spacy
nlp = spacy.load('/content/drive/MyDrive/NER_RE_New/NER/output/model-best/')
text = ['''HON'BLE THE CHIEF JUSTICE MR. AJAY RASTOGI                                      Order  15/03/2018          This review application has been filed at the instance of the  claimant to review and recall the detailed and reasoned judgment  dt. 30.8.2017 passed in MAC APP. No.68/2013 with Cross  Objection (F.A.) No.24/2013.\n\n\n\tThe main thrust of submission of counsel for review  petitioner (claimant) is that after passing of the judgment of this  court on 30.8.2017, the later judgment of the Apex Court in  National Insurance Company Ltd. Vs. Pranay Sethi & others  reported in AIR 2017 SC 5157 was delivered on 31.10.2017 and  that may come to rescue of the claimant for just compensation  and counsel submits that as far as the judgment of the Apex Court  being     binding   under    Article      141    of   the   Constitution,   the  compensation which          has    been awarded to           him   under the  impugned judgment deserves to be reviewed but to his misfortune                                Page 2 of 2   the Apex Court passed judgment at a later point of time and that  is the reason for which the present review petition has been filed.\n\n\n\tThe review permissible under Order XLVII Rule 1 CPC has a  very limited scope and can be filed where there is a discovery of  new and important fact or evidence which could not have been  placed despite due diligence or was not within knowledge or on  account of some mistake or error apparent on the face of record.\n\n\n\tEither of the conditions represented under Order XLVII Rule  1 CPC are not made out and the later judgment of the Apex Court  dt. 31.10.2017 indeed laid down the law binding under Article 141  of the Constitution but that will not open a window for filing a  review application for reappraisal of this view express on the basis  of the law as has been in force at the relevant point of time while  the judgment pronounced by this court.\n\n\tConsequently, in my view, the review petition is without  substance and accordingly dismissed.''']
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print("Entities:")
    print([(ent.start,ent.end,ent.text, ent.label_) for ent in doc.ents])

OSError: ignored

# RE (Relation Extraction) Train

## I/P: .json file with entities and relations (created from UBIAI annotation tool)

In [ ]:
## https://towardsdatascience.com/how-to-train-a-joint-entities-and-relation-extraction-classifier-using-bert-transformer-with-spacy-49eb08d91b5c

In [ ]:
!pip install -U spacy-nightly --pre

     |████████████████████████████████| 12.7 MB 198 kB/s 
     |████████████████████████████████| 9.1 MB 24.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 451 kB 46.4 MB/s 
     |████████████████████████████████| 628 kB 45.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.6 wh

In [ ]:
cd /content/drive/MyDrive/NER_RE_New/RE

/content/drive/MyDrive/NER_RE_New/RE


In [ ]:
!pip install -U pip setuptools wheel

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 946 kB 44.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.6 which is incompatible.
spacy 2.2.4 requires srsly<1.1.0,>=1.0.2, but you have srsly 2.4.2 which is incompatible.
spacy 2.2.4 requires thinc==7.4.0, but you have thinc 8.0.12 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!python -m spacy project clone tutorials/rel_component

✔ Cloned 'tutorials/rel_component' from explosion/projects
/content/drive/MyDrive/NER_RE_New/RE/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/drive/MyDrive/NER_RE_New/RE/rel_component


In [ ]:
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers

     |████████████████████████████████| 459.7 MB 1.4 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.3 MB 4.8 MB/s            
     |████████████████████████████████| 42 kB 22.6 MB/s            
     |████████████████████████████████| 2.6 MB 37.4 MB/s            
     |████████████████████████████████| 42 kB 20.2 MB/s            
     |████████████████████████████████| 40 kB 48.5 MB/s            
     |████████████████████████████████| 998 kB 43.0 MB/s            
     |████████████████████████████████| 895 kB 46.2 MB/s            
     |████████████████████████████████| 3.3 MB 39.4 MB/s            
     |████████████████████████████████| 596 kB 46.4 MB/s            
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-py3-none-any.whl size=459708989 sha256=295aa75b886502ac75e4ca4fa4a27b004bad13c0a1b38691a36c0357d5aab316
  Stored in directory: /tmp/pip-ephem-wheel-cache-jo8tfjd0/wheels/be/d9/92/c65214265163256ecc7588e6

In [ ]:
!mkdir /content/drive/MyDrive/NER_RE_New/RE/rel_component/data
#upload data Train,Dev,Test

In [ ]:
!pwd

/content/drive/My Drive/NER_RE_New/RE


In [ ]:
cd /content/drive/My Drive/NER_RE_New/RE

/content/drive/MyDrive/NER_RE_New/RE


In [ ]:
#On converting the files into spacy if you are getting multiple instance error remove multiple instances of the same token under one document inside the json file.
#If you are getting a None type error on detecting the token entity, remove trailing spaces in the annotated text inside the json file

In [ ]:
## Upload json data Train.json, Test.json, Dev.json created from UBIAI annotation  tool . Use  binary_converter.py from https://github.com/walidamamou/relation_extraction_transformer and convert to spacy
!python3 binary_converter.py

ℹ 5 training sentences
ℹ 96 training sentences
ℹ 5 training sentences


In [ ]:
cd /content/drive/MyDrive/NER_RE_New/RE/rel_component/

/content/drive/MyDrive/NER_RE_New/RE/rel_component


In [ ]:
!pwd

/content/drive/My Drive/NER_RE_New/RE/rel_component


In [ ]:
#refer to this if you are getting a key error like this KeyError: (11, 18) --> https://github.com/explosion/spaCy/discussions/9551

In [ ]:
!spacy project run train_gpu


================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train data/train.spacy --paths.dev data/dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-10-30 05:46:33,176] [INFO] Set up nlp object from config
[2021-10-30 05:46:33,187] [INFO] Pipeline: ['transformer', 'relation_extractor']
[2021-10-30 05:46:33,192] [INFO] Created vocabulary
[2021-10-30 05:46:33,194] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 556kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 5.20MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 3.17MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 7.10MB/s]
Downloading: 100% 478M/478M [00:14<00:00, 35.6MB/s]
Some weights of the model checkpoint at roberta-base were not used when initia

In [ ]:
cd /content/drive/My Drive/NER_RE_New/RE/rel_component

/content/drive/MyDrive/NER_RE_New/RE/rel_component


In [ ]:
!spacy project run evaluate 


================================== evaluate ==================================
Running command: /usr/bin/python3 ./scripts/evaluate.py training/model-best data/test.spacy False

Random baseline:
threshold 0.00 	 {'rel_micro_p': '5.56', 'rel_micro_r': '100.00', 'rel_micro_f': '10.53'}
threshold 0.05 	 {'rel_micro_p': '5.77', 'rel_micro_r': '100.00', 'rel_micro_f': '10.91'}
threshold 0.10 	 {'rel_micro_p': '5.56', 'rel_micro_r': '91.67', 'rel_micro_f': '10.48'}
threshold 0.20 	 {'rel_micro_p': '4.92', 'rel_micro_r': '75.00', 'rel_micro_f': '9.23'}
threshold 0.30 	 {'rel_micro_p': '4.55', 'rel_micro_r': '58.33', 'rel_micro_f': '8.43'}
threshold 0.40 	 {'rel_micro_p': '4.03', 'rel_micro_r': '41.67', 'rel_micro_f': '7.35'}
threshold 0.50 	 {'rel_micro_p': '2.88', 'rel_micro_r': '25.00', 'rel_micro_f': '5.17'}
threshold 0.60 	 {'rel_micro_p': '3.41', 'rel_micro_r': '25.00', 'rel_micro_f': '6.00'}
threshold 0.70 	 {'rel_micro_p': '1.45', 'rel_micro_r': '8.33', 'rel_micro_f': '2.47'}
threshol